# Importações

In [67]:
import pandas as pd
import numpy as np

from copy import copy, deepcopy
from typing import List

# Funções

In [68]:
class Environment:
  def __init__(self, root, rel_path):
    self.root = root            # Pasta raiz
    self.rel_path = rel_path    # Caminho relativo

  def get_file_path(self):
    """
    Retorna caminho do arquivo.
    :return: Caminho do arquivo.
    """
    parts = [self.root]
    parts.extend(self.rel_path)
    return "/".join(parts)

  def clone(self, mode="shallow"):
    """
    Retorna uma cópia do objeto. O padrão é shallow copy.
    :param mode: Modo de cópia ("shallow" ou "deep").
    :return: Cópia do objeto.
    """
    if mode == "deep":
      return deepcopy(self)
    return copy(self)

def df_head(df: pd.DataFrame, num: int = 5) -> None:
  """
  Exibe as primeiras linhas do DataFrame.

  :param df: DataFrame a ser exibido.
  :param num: Número de linhas a serem exibidas.
  """
  display(df.head(num))
  print(f"Shape: {df.shape}")

def df_metrics2df(df: pd.DataFrame, columns: List[str], metrics: List[str] = None):
  """
    Calcula métricas estatísticas para colunas selecionadas de um DataFrame e retorna um novo DataFrame
    com as colunas originais como índices e as métricas como colunas.

    Args:
        df (pd.DataFrame): DataFrame de entrada contendo os dados numéricos.
        columns (List[str]): Lista de nomes de colunas numéricas do DataFrame `df` para calcular as métricas.
        metrics (List[str], optional): Lista de categorias de métricas a calcular.
            Pode conter qualquer combinação dos seguintes valores:
                - "centrality": Inclui média, mediana e moda.
                - "dispersion": Inclui amplitude, variância, desvio padrão e coeficiente de variação.
                - "position": Inclui os quartis Q1, Q2 (mediana) e Q3.
                - "skew": Inclui o coeficiente de assimetria (skewness).
                - "kurtosis": Inclui a curtose.
            Se None, todas as métricas são calculadas.

    Returns:
        pd.DataFrame: DataFrame com as colunas selecionadas como índice (linhas), e as métricas como colunas.

    Observações:
        - A moda pode retornar uma lista (caso haja múltiplos valores com a mesma frequência).
        - O coeficiente de variação é calculado como (desvio padrão / média), retornando NaN se a média for zero.
  """
  if metrics is None:
    metrics = ["centrality", "dispersion", "position", "skew", "kurtosis"]

  result = {}

  for col in columns:
    col_metrics = {}

    # Tendência Central
    if "centrality" in metrics:
      col_metrics["media"] = df[col].mean()
      col_metrics["mediana"] = df[col].median()
      modas = df[col].mode().tolist()
      col_metrics["moda"] = modas if len(modas) > 1 else modas[0] if modas else np.nan

    # Dispersão
    if "dispersion" in metrics:
      col_metrics["amplitude"] = df[col].max() - df[col].min()
      col_metrics["variancia"] = df[col].var()
      col_metrics["desv_pad"] = df[col].std()
      mean = df[col].mean()
      std_dev = df[col].std()
      col_metrics["coef_var"] = std_dev / mean if mean != 0 else pd.NA

    # Posição
    if "position" in metrics:
      col_metrics["q1"] = df[col].quantile(0.25)
      col_metrics["q2"] = df[col].quantile(0.50)
      col_metrics["q3"] = df[col].quantile(0.75)

    # Assimetria
    if "skew" in metrics:
      col_metrics["assimetria"] = df[col].skew()

    # Curtose
    if "kurtosis" in metrics:
      col_metrics["curtose"] = df[col].kurtosis()

    result[col] = col_metrics

  return pd.DataFrame(result).T # Transpõe o DataFrame antes de retornar


def df_classify_vars(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    '''
    Classifica colunas como qualitativas, quantitativas discretas ou contínuas.

    :param df: DataFrame com os dados.
    :param columns: Lista de colunas a classificar.
    :return: DataFrame com nome da coluna, tipo base e classificação estatística.
    '''
    resultado = []

    for col in columns:
        dtype = df[col].dtype
        classificacao = "Qualitativo"

        if pd.api.types.is_integer_dtype(dtype):
            classificacao = "Quantitativo Discreto"
        elif pd.api.types.is_float_dtype(dtype):
            classificacao = "Quantitativo Contínuo"

        resultado.append({
            "coluna": col,
            "dtype": str(dtype),
            "classificacao": classificacao
        })

    return pd.DataFrame(resultado)

def freq_table_qualitativas(df: pd.DataFrame, columns: List[str]) -> dict:
    """
    Gera e imprime tabelas de frequência (absoluta, relativa e acumuladas) para colunas qualitativas,
    com saída formatada.

    :param df: DataFrame com os dados.
    :param columns: Lista de colunas qualitativas.
    :return: Dicionário com DataFrames por coluna.
    """
    tabelas = {}

    for col in columns:
        serie = df[col]
        freq_abs = serie.value_counts().sort_index()
        freq_rel = (freq_abs / freq_abs.sum())
        freq_abs_ac = freq_abs.cumsum()
        freq_rel_ac = freq_rel.cumsum()

        tabela = pd.DataFrame({
            "Frequência Absoluta": freq_abs,
            "Frequência Relativa (%)": (freq_rel * 100),
            "Freq. Absoluta Acumulada": freq_abs_ac,
            "Freq. Relativa Acumulada (%)": (freq_rel_ac * 100)
        })

        tabelas[col] = tabela

        # Impressão formatada
        print(f"\nTabela de Frequência – Coluna: **{col}**")
        print(tabela.to_string())





# Setup do Ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

# Obtenção do Dataset

In [ ]:
# Ambiente de entrada
ROOT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked/_all"
FILE = "all_merged.csv"
env_in = Environment(ROOT, [FILE])

df = pd.read_csv(env_in.get_file_path())

df_head(df)

# Classificação dos Atributos

Classifiquemos as variáveis do _dataset_ de acordo com seu tipo e com sua cardinalidade segundo as definições de Han _et al_ (2011):

As variáveis podem ser:



1.  `_ano`: quantitativo discreto
2.  `estado`: qualitativo nominal
3.  `_mes`: quantitativo discreto
4.  `car_c02_emitido`: quantitativo contínuo
5.  `cli_pressao_atm_med`: quantitativo contínuo
6.  `cli_temp_ar_med`: quantitativo contínuo
7.  `cli_temp_orvalho_med`: quantitativo contínuo
8.  `cli_umid_rel_med`: quantitativo contínuo
9.  `cli_umid_rel_min_max`: quantitativo contínuo
10. `cli_umid_rel_min_med`: quantitativo contínuo
11. `cli_umid_rel_min_min`: quantitativo contínuo
12. `cli_veloc_vento_max`: quantitativo contínuo
13. `cli_veloc_vento_med`: quantitativo contínuo
14. `que_area_queimada`: quantitativo contínuo
15. `que_focos_qtd`: quantitativo contínuo

In [69]:
df = pd.read_csv("all_merged.csv")
df_head(df)

list_col = df.columns.tolist()

df_classify_vars(df, list_col)


,_ano,_estado,_mes,car_c02_emitido,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_max,cli_umid_rel_min_med,cli_umid_rel_min_min,cli_veloc_vento_max,cli_veloc_vento_med,que_area_queimada,que_focos_qtd
0,2008,AC,7,2.627698e+15,986.843612,28.142731,18.914978,59.555066,95.0,54.432558,29.0,5.1,2.152915,4957.0,165.0
1,2008,AC,9,2.627698e+15,991.705941,24.446194,19.467987,75.811881,97.0,72.496700,25.0,1.0,0.210504,46073.0,2947.0
2,2008,AC,10,2.627698e+15,990.328360,25.229298,21.617473,81.870968,96.0,78.720430,29.0,1.0,0.204959,30355.0,856.0
3,2008,AC,11,2.627698e+15,988.610987,25.195410,22.624478,86.905424,96.0,84.048679,42.0,1.0,0.186970,2082.0,63.0
4,2008,AC,12,2.627698e+15,988.692608,24.898790,22.727554,88.529570,96.0,86.116935,53.0,1.0,0.179442,127.0,4.0


Shape: (1025, 15)


,coluna,dtype,classificacao
0,_ano,int64,Quantitativo Discreto
1,_estado,object,Qualitativo
2,_mes,int64,Quantitativo Discreto
3,car_c02_emitido,float64,Quantitativo Contínuo
4,cli_pressao_atm_med,float64,Quantitativo Contínuo
5,cli_temp_ar_med,float64,Quantitativo Contínuo
6,cli_temp_orvalho_med,float64,Quantitativo Contínuo
7,cli_umid_rel_med,float64,Quantitativo Contínuo
8,cli_umid_rel_min_max,float64,Quantitativo Contínuo
9,cli_umid_rel_min_med,float64,Quantitativo Contínuo


#Cálculo de Medidas Estatísticas de Dados Quantitativos

In [70]:
list_col = df.select_dtypes(include=["number"]).columns.tolist()
df_metrics2df(df, list_col)

,media,mediana,moda,amplitude,variancia,desv_pad,coef_var,q1,q2,q3,assimetria,curtose
_ano,2014.468293,2014.0,"[2015, 2019]",23,27.967988,5.288477,0.002625,2010.0,2014.0,2019.0,-0.200539,-0.680608
_mes,6.500488,7.0,"[1, 5]",11,11.927978,3.453691,0.531297,4.0,7.0,9.0,-0.004483,-1.214559
car_c02_emitido,13249660172165332.0,5911617948717380.0,"[609293347563185.0, 811256602162324.0, 1268535...",65557202664920592.0,242115245125666742589026534948864.0,15560052863845506.0,1.174374,4228806597067769.5,5911617948717380.0,13033552531398800.0,1.798417,2.285938
cli_pressao_atm_med,993.395296,997.229524,"[1007.8279069767442, 1011.1125]",89.800611,292.255684,17.095487,0.017209,986.19747,997.229524,1004.589771,-2.160808,5.872654
cli_temp_ar_med,26.429096,26.29328,"[21.97972222222222, 21.99903181189488, 22.0466...",9.783472,2.106806,1.451484,0.05492,25.463194,26.29328,27.304131,0.369335,0.56568
cli_temp_orvalho_med,21.282504,22.111877,"[20.767185289957567, 22.77238493723849]",18.756593,7.506876,2.739868,0.128738,20.737032,22.111877,22.931942,-2.164032,5.28379
cli_umid_rel_med,75.905264,79.0,83.438172,75.420653,135.841085,11.655088,0.153548,70.405995,79.0,83.946019,-1.366295,2.001775
cli_umid_rel_min_max,94.459512,95.0,95.0,70.0,26.313056,5.129625,0.054305,94.0,95.0,97.0,-5.131046,45.514632
cli_umid_rel_min_med,72.934699,75.870968,"[71.88888888888889, 81.89516129032258]",76.898499,142.161788,11.923162,0.163477,67.305556,75.870968,81.094891,-1.274984,1.74467
cli_umid_rel_min_min,38.312195,39.0,34.0,68.0,161.20908,12.696814,0.331404,30.0,39.0,47.0,-0.096313,-0.050843


#Tabela de Frequência

In [71]:
list_col_qualitativo = ["_estado"]

freq_table_qualitativas(df, list_col_qualitativo)


Tabela de Frequência – Coluna: **_estado**
         Frequência Absoluta  Frequência Relativa (%)  Freq. Absoluta Acumulada  Freq. Relativa Acumulada (%)
_estado                                                                                                      
AC                       113                11.024390                       113                     11.024390
AM                       183                17.853659                       296                     28.878049
AP                       114                11.121951                       410                     40.000000
PA                       156                15.219512                       566                     55.219512
RO                       134                13.073171                       700                     68.292683
RR                       121                11.804878                       821                     80.097561
TO                       204                19.902439                      1

# Referências

HAN, Jiawei; KAMBER, Micheline; PEI, Jian. **Data mining: concepts and techniques**. 3. ed. Burlington: Morgan Kaufmann, 2011.